### 实验目的：
使用一站式在线平台搭建神经网络，实现深度学习算法的网络组建、训练、预测过程
### 实验要求：
完成给定数据集上的分类任务，定义全连接前馈神经网络，softmax输出层以及损失函数 ，训练网路，使用给定的数据集中的测试数据验证精度。
### 基本的实验数据集
MNIST数据集 Cifar10数据集等

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import pickle

 ### 搭建神经网络
（1）加载数据，引入相关的包

（2）根据网络结构，定义各参数、变量，这是最重要的一步，搭建好网络结构就只需要喂数据

（3）搭建好网络结构，提供数据进行训练并保存训练模型

（4）用数据训练好模型

（5）模型评价

#### 搭建网络结构

1. keras.Sequential() 函数创建顺序模型
2. model.add函数来添加网络结构的层数，简单网络一般是输入层，隐含层，输出层三层结构
3. keras.layers.Dense 函数创建全连接的网络，有两个关键的参数，一个是神经元的个数units,一个是激活函数的类型activation，注意:每一层神经元的个数是上一层的输出个数
4. keras.layers.Conv2D 创建卷积层，过滤器filters,卷积核的大小kernel_size,边缘处理padding,激活函数activation
5. keras.layers.MaxPool2D 创建完卷积层，然后就是用池化层，这里用最大池化和平均池化，pool_size池化的大小
6. 最后还是一样，一个全连接的输出层
* 训练模型

In [ ]:

# 搭建 简单 模型 
def build_simple_model(out_dims, img_size):
    inputs_dim = Input((img_size, img_size, 3))

    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same',activation='relu')(inputs_dim)
    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same',activation='relu')(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

    x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',activation='relu')(x)
    x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',activation='relu')(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

    x_flat = Flatten()(x)
    fc1 = Dense(512,activation='relu')(x_flat)
    dp_1 = Dropout(0.4)(fc1)

    fc2 = Dense(out_dims)(dp_1)
    fc2 = Activation('softmax')(fc2)

    model = Model(inputs=inputs_dim, outputs=fc2)
    return model

def load_CIFAR_batch(filename):
    with open(filename, 'rb') as f:
        datadict = pickle.load(f,encoding='latin1')
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32,32).transpose(0,2,3,1).astype("float") #channel last
        Y = np.array(Y)
    return X, Y

def load_CIFAR10(ROOT):
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

ROOT = '/kaggle/input/dataset-cifar/cifar-10-batches-py'
X_train,y_train,X_test,y_test = load_CIFAR10(ROOT)

X_train /= 255
X_test /= 255
y_train = np_utils.to_categorical(y_train, num_classes = 10)
y_test = np_utils.to_categorical(y_test, num_classes = 10)

X_train = X_train.astype(np.float32) #'float32'占4个字节
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.uint8) # 'uint8'占1个字节
y_test = y_test.astype(np.uint8)

X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=0)


simple_model = build_simple_model(10,32)
simple_model.summary()

#### 网络结构搭建完后 ，开始训练模型
1. 在这之前可以对数据做一些处理，正则化等操作这里用了标准化数据
2. np.expand_dims 看成 增加维度，变化数据的维度
3. 标签需要one_hot，使用onehot的直接原因是现在多分类cnn网络的输出通常是softmax层，而它的输出是一个概率分布，从而要求输入的标签也以概率分布的形式出现，进而算交叉熵之类。
4. 配置训练模型。optimizer优化器，loss损失函数，学习率，迭代次数等参数
* 最后开始训练

In [2]:
checkpointer = ModelCheckpoint(filepath='cifa_10_simple.hdf5', 
                              verbose=1, save_best_only=True) #保存最好模型权重
simple_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 5

history = simple_model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=epochs,callbacks=[checkpointer],verbose=1)

NameError: name 'ModelCheckpoint' is not defined

### 性能评估 

用开始创建的类history绘图的acc和loss变化情况，可以看每一层迭代时的变化情况





**参数对性能的影响:**

可参考 ：https://blog.csdn.net/program_developer/article/details/78597738

这里是举几个参数说明:

激活函数： 

Sigmoid函数：常用的非线性的激活函数，它能够把输入的连续实值变换为0和1之间的输出，特别的，如果是非常大的负数，那么输出就是0；如果是非常大的正数，输出就是1.

tanh函数：解决了Sigmoid函数的不是zero-centered输出问题

Relu函数：ReLU目前仍是最常用的activation function，在搭建人工神经网络的时候推荐优先尝试

迭代次数：epochs（epochs指的就是训练过程中数据将被“轮”多少次）

训练的数量: Batch Size（batch size将决定我们一次训练的样本数目）


这里使用ModelCheckpoint 记录中间训练的过程

最后绘制 acc 和 loss 的变化情况


In [1]:
import matplotlib.pyplot as plt                        
# %matplotlib inline    

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined